In [1]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output

Game = Regency()
clear_output()
for a in range(1):
    Game.random_events()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_regency_points()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.domain_initiative()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_gold_bars()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.maintenance_costs()
    print(len((Game.Seasons[Game.Season]['Season'])))

Game.clear_screen()
self 

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,JR,False,No Event,33,21,-1,2,3,Dormant
1,TCV,False,Intrigue,1,20,0,1,1,Dormant
2,CSH,False,Intrigue,10,18,3,6,3,Bare
3,Sw2,False,Diplomatic Matter,24,17,1,0,2,Bare
4,MF,False,No Event,59,17,26,7,9,Rich
5,LPA,False,Feud,11,14,5,9,4,Bare
6,WM,False,Corruption/Crime,7,13,1,2,1,Dormant
7,GH,False,No Event,10,13,6,9,3,Bare
8,IHH,False,No Event,6,12,2,5,3,Bare
9,Sw,False,Assassination,11,12,0,0,0,Dormant


In [63]:
import pandas as pd
self = Game
df = self.Seasons[self.Season]['Season'][self.Seasons[self.Season]['Season']['Initiative'] == 13].copy()
for i, row_ in df.reset_index().iterrows():
    if i == 0:
        row = row_
        
# start

# get nearby regents
temp = self.Provences[self.Provences['Regent'] == row['Regent']]['Provence']
temp = pd.concat([temp, self.Holdings[self.Holdings['Regent'] == row['Regent']]['Provence']])
# temp is a provence list
pd.merge(temp, self.Geography[self.Geography['Border']==1][['Provence','Neighbor']], on='Provence', how='left')['Neighbor']


,Provence,Neighbor
0,Moergen,Ghalliere
1,Moergen,Gulfport
2,Moergen,Moriel
3,Moergen,Sunken Lands
4,Moergen,Bogsend
5,Moriel,Ghalliere
6,Moriel,Gulfport
7,Moriel,Brothendar
8,Moriel,Moergen
9,Moriel,Spiritsend


In [47]:
self.Relationships

,Regent,Other,Diplomacy,Payment,Vassalage,At War,Trade Permission
0,Sw2,WM,0,0,2,0,0
1,LPA,EC,3,0,1,0,0
2,JR,WM,-2,0,0,0,0
3,WM,JR,-3,0,0,0,0
4,MF,GG,10,10,2,0,0
5,DW,GG,13,10,3,0,0
6,TBM,Ch,-1,0,0,0,0
7,Col,Ch,-1,0,1,0,0
8,Is,GG,3,0,0,0,0
